In [15]:
import getpass, requests, io
import pandas as pd
import warnings
from azure_connector import Azure_connector

connector = Azure_connector()

warnings.filterwarnings('ignore')
 
def lthelper_dl(passwd, countries):
    wp_login = "https://lantools.it-solutions.myatos.net/wp-login.php"
    wp_admin = "https://lantools.it-solutions.myatos.net/wp-admin/"

    output = pd.DataFrame()
    for country in countries:
        with requests.Session() as wordpress:
            try:
                headers = {'Cookie': 'wordpress_test_cookie=WP Cookie check'}
                datas = {'log': username, 'pwd': passwd, 'wp-submit': 'Log In', 'redirect_to': wp_admin, 'testcookie': '1'}
                wordpress.post(wp_login, headers=headers, data=datas, verify=False)
                wordpress.get(wp_admin)

                llink = f"https://lantools.it-solutions.myatos.net/reports/auswertungen/php/report_export_csv.php?&search={country};%;;;;BASIC;5846"
                
                dl = wordpress.get(llink, verify=False, timeout=600).content
                
                df = pd.read_csv(io.StringIO(dl.decode('utf-8')), 
                                 on_bad_lines='skip', 
                                 sep=';',
                                 usecols=['Country', 'Location', 'Switch',
                        'SwitchIP', 'SlotPort', 'SGPort', 'SGVLAN', 'SGVLANName',
                        'SGLastActivityPort', 'VoiceVLAN', 'RadiusMACVoice', 'RadiusDateVoice',
                        'SGPhysAddr with stat IP in GT', 'GTFQDN', 'Description', 'SGPhysAddr',
                        'SGNetAddr', 'SGLastActivityMAC', 'GTFirstName', 'GTName',
                        'GTDepartment', 'GTGID', 'SGKonfGBPort', 'SGMaxCapUsed', 'SGCustomer',
                        'Vendor', 'SGPortStatus', 'confVLAN', 'confVLANName', 'Unique ID Port',
                        'Port Name', 'PoE AdStatus', 'PoE Status','Dot1x', 'LLDP Topology', 'Type of port','Logical network of the switch'])
                
                print(f"{country}: {df.shape[0]} records")
                
                output = pd.concat([output,df],ignore_index=True) 

            except requests.RequestException as e:
                print("Wyjątek:", str(e))
            # # # # except:
                print("Unknown ERROR")
                
    return output

countries = ['DE']

username = getpass.getuser()
password = getpass.getpass(prompt="SaACon Password: ")
df = lthelper_dl(passwd=password, countries=countries)  # SaACon Password

df.columns = [i.lower().replace(" ","_") for i in df.columns]
df.description.fillna("none", inplace=True)
df.description = df.description.str.lower()


DE: 354326 records


In [16]:
df.columns = [i.lower().replace(" ","_") for i in df.columns]
df.description.fillna("none", inplace=True)
df.description = df.description.str.lower()
import re

def category(df):
    if df.sgmaxcapused == "10Gbps":
        if re.search(".*snx|.*wan", df.description): category = "wan"
        elif re.search("^(tr|s-|r-)|.*upl|.*down", df.description): category = "link"
        elif re.search(".*fw", df.description): category = "fw"
        elif re.search(".*ncs", df.description): category = "ncs"
        elif re.search(".*ofc|.*office", df.description): category = "office"
        elif re.search(".*rvlan", df.description): category = "rvlan"
        elif re.search(".*srv|.*server|.*esx", df.description): category = "server"
        elif re.search(".*wlan|^w-|.*-lc$", df.description): category = "wlc"
        else: category = "other"  
    else:
        category = None       
    return category

df["tengbit_catgory"] = df.apply(category, axis=1)
df.tengbit_catgory.value_counts(dropna=False)

df["tengbit_count"] = df.groupby("switch")["sgmaxcapused"].transform(lambda x: x.str.contains("10Gbps").sum())
df["wan_count"] = df.groupby("switch")["tengbit_catgory"].transform(lambda x: x.str.contains("wan").sum())
df = df.loc[df.tengbit_count >0, :]
ru30 = connector.query("select u_fqdn as fqdn, u_ru_current_month as ru from [DW_SNOWMIRROR_GL].[u_siemens_mmsa_order_book] where u_ru_current_month = 'CU3989' and u_recurring_entry = 'Yes'")

df = df.merge(ru30, left_on="port_name", right_on="fqdn", how='left')
del df["fqdn"]

df["ru30_count"] = df.groupby("switch")["ru"].transform(lambda x: x.count())


In [47]:
spath = f"Z:\\_protected\\A605716\\ru30_analysis.xlsx"
df.to_excel(spath)

In [46]:
df

,country,location,switch,switchip,slotport,sgport,sgvlan,sgvlanname,sglastactivityport,voicevlan,...,poe_adstatus,poe_status,dot1x,lldp_topology,type_of_port,tengbit_catgory,tengbit_count,ru,wan_count,ru30_count
0,DE,ABG MB 5,s-de-abgmb5-c-eg-com-2-poe,139.16.192.180,GigabitEthernet1/0/11,11.0,666.0,VL-G-DE-UP14-0001,NaN,unknown,...,disable,disabled,not set,NaN,NaN,None,1,NaN,0,0
1,DE,ABG MB 5,s-de-abgmb5-c-eg-com-2-poe,139.16.192.180,GigabitEthernet1/0/12,12.0,666.0,VL-G-DE-UP14-0001,NaN,unknown,...,disable,disabled,not set,NaN,NaN,None,1,NaN,0,0
2,DE,ABG MB 5,s-de-abgmb5-c-eg-com-2-poe,139.16.192.180,GigabitEthernet1/0/13,13.0,666.0,VL-G-DE-UP14-0001,NaN,unknown,...,disable,disabled,not set,NaN,NaN,None,1,NaN,0,0
3,DE,ABG MB 5,s-de-abgmb5-c-eg-com-2-poe,139.16.192.180,GigabitEthernet1/0/14,14.0,666.0,VL-G-DE-UP14-0001,NaN,unknown,...,disable,disabled,not set,NaN,NaN,None,1,NaN,0,0
4,DE,ABG MB 5,s-de-abgmb5-c-eg-com-2-poe,139.16.192.180,GigabitEthernet1/0/15,15.0,666.0,VL-G-DE-UP14-0001,NaN,unknown,...,disable,disabled,not set,NaN,NaN,None,1,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221893,DE,WOB D,r-de-wobd-0-1og-206-1-cor,129.103.111.121,TenGigabitEthernet1/1/6,6.0,1.0,default,NaN,unknown,...,NaN,NaN,not set,NaN,NaN,None,1,NaN,0,0
221894,DE,WOB D,r-de-wobd-0-1og-206-1-cor,129.103.111.121,TenGigabitEthernet1/1/7,7.0,1.0,default,NaN,unknown,...,NaN,NaN,not set,NaN,NaN,None,1,NaN,0,0
221895,DE,WOB D,r-de-wobd-0-1og-206-1-cor,129.103.111.121,TenGigabitEthernet1/1/8,8.0,-1.0,NaN,21.02.2024,unknown,...,NaN,NaN,not set,NaN,NaN,None,1,NaN,0,0
221896,DE,WOB D,r-de-wobd-0-1og-206-1-cor,129.103.111.121,TwentyFiveGigE1/1/1,1.0,1.0,default,NaN,unknown,...,NaN,NaN,not set,NaN,NaN,None,1,NaN,0,0


In [44]:
a.count()

618